# Data Import and Cleaning

This notebook demonstrate the process of importing historical data on CAC 40 firm's stock prices and french treasury bonds. Some packages are required to run this code which can be found in the `requirements.txt` text file.

**Table of content :**

1. Importing Data
2. Data Cleanup

## Importing Data

### CAC 40 Historical Prices

*Note: The components of the CAC 40 have evolved over time. In the case of this work, the list of firms studied are took from the latest change to the index as of writing, which was on september 30th 2024. The detailled list can be found in the `doc` folder of this project.*

Start by importing the required packages for importing stock data and creating dataframes.

In [726]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
import plotly.express as px
import xml.etree.ElementTree as ET

If the packages aren't already installed, they can be installed in this environment by using these commands.

In [729]:
# %pip install yfinance
# %pip install plotly

Defining functions for importing the dataset. The first function call the Yahoo Finance API to gather monthly historical data on a given ticker within a given period. The second function iterate through a list to activate the first function for each ticker. A delay of one second is executed after each successful import attempt to prevent getting rate-limited by the API.

In [732]:
def api_request(x, a, b, c):
    x = yf.Ticker(x)
    y = x.history(start=a, end=b, interval=c)
    return y

def get_stocks(x, a, b, c, d):
    l = len(x)
    y = []
    for i, row in x.iterrows():
        n = row["Name"]
        s = row["Sector"]
        j = row["Ticker"]
        print(f"[{i + 1}/{l}] {j}...", end="")
        z = api_request(j, a, b, c)
        z = z.reset_index()
        z.insert(0, "Ticker", j)
        z.insert(1, "Name", n)
        z.insert(2, "Sector", s)
        y.append(z)
        print(" X")
        time.sleep(d)
    y = pd.concat(y, ignore_index=True)
    y["Date"] = pd.to_datetime(y["Date"], utc=True).dt.strftime('%Y-%m')
    return y

Import the dataset for the given timeframe and build a database with the stock prices. **The program can take up to a few minutes to successfully import the data.**

In [735]:
tickers = pd.read_csv("data/raw/CAC40.csv", sep=";")
cac = get_stocks(tickers, "2008-12-01", "2024-02-01", "1mo", 1)

cac

[1/40] AC.PA... X
[2/40] AI.PA... X
[3/40] AIR.PA... X
[4/40] MT.AS... X
[5/40] CS.PA... X
[6/40] BNP.PA... X
[7/40] EN.PA... X
[8/40] CAP.PA... X
[9/40] CA.PA... X
[10/40] ACA.PA... X
[11/40] BN.PA... X
[12/40] DSY.PA... X
[13/40] EDEN.PA... X
[14/40] ENGI.PA... X
[15/40] EL.PA... X
[16/40] ERF.PA... X
[17/40] RMS.PA... X
[18/40] KER.PA... X
[19/40] OR.PA... X
[20/40] LR.PA... X
[21/40] MC.PA... X
[22/40] ML.PA... X
[23/40] ORA.PA... X
[24/40] RI.PA... X
[25/40] PUB.PA... X
[26/40] RNO.PA... X
[27/40] SAF.PA... X
[28/40] SGO.PA... X
[29/40] SAN.PA... X
[30/40] SU.PA... X
[31/40] GLE.PA... X
[32/40] STLAP.PA... X
[33/40] STMPA.PA... X
[34/40] TEP.PA... X
[35/40] HO.PA... X
[36/40] TTE.PA... X
[37/40] URW.PA... X
[38/40] VIE.PA... X
[39/40] DG.PA... X
[40/40] VIV.PA... X


,Ticker,Name,Sector,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,AC.PA,Accor,Consumer Discretionary,2008-11,14.629791,17.635233,13.909480,17.441494,32193151,0.0,0.0
1,AC.PA,Accor,Consumer Discretionary,2008-12,17.441496,18.628769,13.934318,15.397298,58382027,0.0,0.0
2,AC.PA,Accor,Consumer Discretionary,2009-01,15.131527,17.198078,13.760449,14.123089,37496475,0.0,0.0
3,AC.PA,Accor,Consumer Discretionary,2009-02,13.862285,14.505599,12.518529,13.022749,41145268,0.0,0.0
4,AC.PA,Accor,Consumer Discretionary,2009-03,12.955689,16.236836,12.896076,16.005838,42630279,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,8.204041,8.303449,7.831750,8.085141,39145067,0.0,0.0
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,8.432094,8.469127,7.983786,8.241076,37809876,0.0,0.0
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,8.248871,8.628959,8.170905,8.465229,28765845,0.0,0.0
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,8.480822,9.804307,8.356076,9.430066,55547891,0.0,0.0


Get the benchmark.

In [893]:
bk = api_request("^FCHI", "2008-12-01", "2024-02-01", "1mo")
k = pd.DataFrame()
k["Benchmark"] = bk["Close"]
k = k.reset_index()
k["Date"] = pd.to_datetime(k["Date"], utc=True).dt.strftime('%Y-%m')

cac40 = pd.merge(cac, k, on="Date", how="left")
del bk
cac40

,Ticker,Name,Sector,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Benchmark
0,AC.PA,Accor,Consumer Discretionary,2008-11,14.629791,17.635233,13.909480,17.441494,32193151,0.0,0.0,3217.969971
1,AC.PA,Accor,Consumer Discretionary,2008-12,17.441496,18.628769,13.934318,15.397298,58382027,0.0,0.0,2973.919922
2,AC.PA,Accor,Consumer Discretionary,2009-01,15.131527,17.198078,13.760449,14.123089,37496475,0.0,0.0,2702.479980
3,AC.PA,Accor,Consumer Discretionary,2009-02,13.862285,14.505599,12.518529,13.022749,41145268,0.0,0.0,2807.340088
4,AC.PA,Accor,Consumer Discretionary,2009-03,12.955689,16.236836,12.896076,16.005838,42630279,0.0,0.0,3159.850098
...,...,...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,8.204041,8.303449,7.831750,8.085141,39145067,0.0,0.0,7135.060059
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,8.432094,8.469127,7.983786,8.241076,37809876,0.0,0.0,6885.649902
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,8.248871,8.628959,8.170905,8.465229,28765845,0.0,0.0,7310.770020
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,8.480822,9.804307,8.356076,9.430066,55547891,0.0,0.0,7543.180176


### Bons du Trésor

The program keeps only the first and second columns, namely, the date and the average yield of french treasury bonds with a maturity of 13 weeks issued during this month.

In [897]:
tmb = pd.read_csv("https://webstat.banque-france.fr/export/csv-columns/fr/selection/5385695", skiprows=6, sep=";", header=None)
tmb = tmb[~tmb.isin(["-"]).any(axis=1)]
tmb = tmb[[0, 1]]
tmb.rename(columns={0: "Date", 1: "Bond Yield"}, inplace=True)
tmb["Date"] = pd.to_datetime(tmb["Date"]).dt.strftime('%Y-%m')
tmb["Bond Yield"] = tmb["Bond Yield"].str.replace(',', '.').astype(float)

tmb

,Date,Bond Yield
0,2024-11,3.09
1,2024-10,3.17
2,2024-09,3.52
3,2024-08,3.59
4,2024-07,3.76
...,...,...
487,1984-04,NaN
488,1984-03,NaN
489,1984-02,NaN
490,1984-01,NaN


The two dataframes are merged to form the database that will be the basis of this analysis.

In [900]:
data = pd.merge(cac40, tmb, on="Date", how="left")
del tmb

data = data.drop(columns=["Open", "High", "Low", "Volume", "Stock Splits"])
data.rename(columns={"Close": "Price"}, inplace=True)
data

,Ticker,Name,Sector,Date,Price,Dividends,Benchmark,Bond Yield
0,AC.PA,Accor,Consumer Discretionary,2008-11,17.441494,0.0,3217.969971,2.70
1,AC.PA,Accor,Consumer Discretionary,2008-12,15.397298,0.0,2973.919922,2.06
2,AC.PA,Accor,Consumer Discretionary,2009-01,14.123089,0.0,2702.479980,1.32
3,AC.PA,Accor,Consumer Discretionary,2009-02,13.022749,0.0,2807.340088,0.99
4,AC.PA,Accor,Consumer Discretionary,2009-03,16.005838,0.0,3159.850098,0.74
...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,8.085141,0.0,7135.060059,3.68
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,8.241076,0.0,6885.649902,3.85
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,8.465229,0.0,7310.770020,3.89
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,9.430066,0.0,7543.180176,3.89


### Indice des Prix à la Consommation (IPC)

Serving as measure of inflation, produce log inflation.

$$
\text{Log Inflation}_t = \ln \left( \frac{CPI_t}{CPI_{t-1}} \right)
$$

In [903]:
inf = pd.read_csv("data/raw/IPC.csv", skiprows=4, sep=";", header=None)
inf = inf[[0, 1]]
inf.rename(columns={0: "Date", 1: "IPC"}, inplace=True)
inf["Date"] = pd.to_datetime(inf["Date"]).dt.strftime('%Y-%m')
inf = inf.sort_values(by="Date")

inf["Log Inflation"] = np.log(inf["IPC"] / inf["IPC"].shift(1))
inf = inf.drop(columns=["IPC"])
inf

,Date,Log Inflation
214,2007-01,NaN
213,2007-02,0.001890
212,2007-03,0.004323
211,2007-04,0.004965
210,2007-05,0.002528
...,...,...
4,2024-07,0.001929
3,2024-08,0.005347
2,2024-09,-0.012662
1,2024-10,0.002781


In [905]:
data = pd.merge(data, inf, on="Date", how="left")
del inf

data

,Ticker,Name,Sector,Date,Price,Dividends,Benchmark,Bond Yield,Log Inflation
0,AC.PA,Accor,Consumer Discretionary,2008-11,17.441494,0.0,3217.969971,2.70,-0.004796
1,AC.PA,Accor,Consumer Discretionary,2008-12,15.397298,0.0,2973.919922,2.06,-0.002460
2,AC.PA,Accor,Consumer Discretionary,2009-01,14.123089,0.0,2702.479980,1.32,-0.004186
3,AC.PA,Accor,Consumer Discretionary,2009-02,13.022749,0.0,2807.340088,0.99,0.003864
4,AC.PA,Accor,Consumer Discretionary,2009-03,16.005838,0.0,3159.850098,0.74,0.001927
...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,8.085141,0.0,7135.060059,3.68,0.010136
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,8.241076,0.0,6885.649902,3.85,-0.005353
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,8.465229,0.0,7310.770020,3.89,0.001447
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,9.430066,0.0,7543.180176,3.89,-0.001788


### Indice Brut de la Production Industrielle (IBPI)

In [908]:
ibpi = pd.read_csv("data/raw/IBPI.csv", skiprows=4, sep=";", header=None)
ibpi = ibpi[[0, 1]]
ibpi.rename(columns={0: "Date", 1: "IPI"}, inplace=True)
ibpi["Date"] = pd.to_datetime(ibpi["Date"]).dt.strftime('%Y-%m')
ibpi = ibpi.sort_values(by="Date")

ibpi["IPI"] = np.log(ibpi["IPI"] / ibpi["IPI"].shift(1))
ibpi.rename(columns={"IPI": "Log IPI"}, inplace=True)

ibpi

,Date,Log IPI
213,2007-01,NaN
212,2007-02,0.018811
211,2007-03,0.063136
210,2007-04,-0.047514
209,2007-05,-0.097943
...,...,...
4,2024-06,0.123257
3,2024-07,0.017737
2,2024-08,-0.615316
1,2024-09,0.574730


In [910]:
data = pd.merge(data, ibpi, on="Date", how="left")
del ibpi

data

,Ticker,Name,Sector,Date,Price,Dividends,Benchmark,Bond Yield,Log Inflation,Log IPI
0,AC.PA,Accor,Consumer Discretionary,2008-11,17.441494,0.0,3217.969971,2.70,-0.004796,-0.185435
1,AC.PA,Accor,Consumer Discretionary,2008-12,15.397298,0.0,2973.919922,2.06,-0.002460,0.163142
2,AC.PA,Accor,Consumer Discretionary,2009-01,14.123089,0.0,2702.479980,1.32,-0.004186,-0.207272
3,AC.PA,Accor,Consumer Discretionary,2009-02,13.022749,0.0,2807.340088,0.99,0.003864,0.059060
4,AC.PA,Accor,Consumer Discretionary,2009-03,16.005838,0.0,3159.850098,0.74,0.001927,0.044075
...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,8.085141,0.0,7135.060059,3.68,0.010136,-0.390923
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,8.241076,0.0,6885.649902,3.85,-0.005353,0.488359
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,8.465229,0.0,7310.770020,3.89,0.001447,0.104945
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,9.430066,0.0,7543.180176,3.89,-0.001788,-0.108930


### USD to EUR

In [913]:
usd = ET.parse("data/raw/USD.xml")

# Get the root element
r = usd.getroot()

n = {
    "message": "http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message",
    "exr": "http://www.ecb.europa.eu/vocabulary/stats/exr/1"
}

ob = r.find(".//exr:DataSet", n).find(".//exr:Series", n).findall("exr:Obs", n)

usdeur = []
for i in ob:
    a = i.get("TIME_PERIOD")
    b = i.get("OBS_VALUE")
    usdeur.append({"TIME_PERIOD": a, "OBS_VALUE": float(b)})

usd = pd.DataFrame(usdeur)
del usdeur
del a
del b
del ob
del n
del r

usd.rename(columns={"TIME_PERIOD": "Date", "OBS_VALUE": "USD/EUR"}, inplace=True)

usd

,Date,USD/EUR
0,1999-01-04,1.1789
1,1999-01-05,1.1790
2,1999-01-06,1.1743
3,1999-01-07,1.1632
4,1999-01-08,1.1659
...,...,...
6647,2024-12-13,1.0518
6648,2024-12-16,1.0498
6649,2024-12-17,1.0497
6650,2024-12-18,1.0496


In [915]:
usd["Date"] = pd.to_datetime(usd["Date"])
usd["Date"] = usd["Date"].dt.to_period("M")
usd = usd.groupby("Date", as_index=False)['USD/EUR'].mean()
usd["Date"] = usd["Date"].astype(str)
usd["Date"] = pd.to_datetime(usd["Date"]).dt.strftime('%Y-%m')
usd = usd.sort_values(by="Date")

usd["USD/EUR"] = np.log(usd["USD/EUR"] / usd["USD/EUR"].shift(1))
usd.rename(columns={"USD/EUR": "Log USD/EUR"}, inplace=True)
usd

,Date,Log USD/EUR
0,1999-01,NaN
1,1999-02,-0.035081
2,1999-03,-0.029399
3,1999-04,-0.016542
4,1999-05,-0.007155
...,...,...
307,2024-08,0.015382
308,2024-09,0.008483
309,2024-10,-0.018324
310,2024-11,-0.025468


In [917]:
data = pd.merge(data, usd, on="Date", how="left")
del usd

data

,Ticker,Name,Sector,Date,Price,Dividends,Benchmark,Bond Yield,Log Inflation,Log IPI,Log USD/EUR
0,AC.PA,Accor,Consumer Discretionary,2008-11,17.441494,0.0,3217.969971,2.70,-0.004796,-0.185435,-0.045293
1,AC.PA,Accor,Consumer Discretionary,2008-12,15.397298,0.0,2973.919922,2.06,-0.002460,0.163142,0.054806
2,AC.PA,Accor,Consumer Discretionary,2009-01,14.123089,0.0,2702.479980,1.32,-0.004186,-0.207272,-0.015795
3,AC.PA,Accor,Consumer Discretionary,2009-02,13.022749,0.0,2807.340088,0.99,0.003864,0.059060,-0.034893
4,AC.PA,Accor,Consumer Discretionary,2009-03,16.005838,0.0,3159.850098,0.74,0.001927,0.044075,0.020525
...,...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,8.085141,0.0,7135.060059,3.68,0.010136,-0.390923,-0.013591
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,8.241076,0.0,6885.649902,3.85,-0.005353,0.488359,-0.020847
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,8.465229,0.0,7310.770020,3.89,0.001447,0.104945,-0.011415
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,9.430066,0.0,7543.180176,3.89,-0.001788,-0.108930,0.022993


## Cleaning Data

Transforming stock price to log return.

$$
r_i,t = \ln \left( \frac{P_{i,t} + D_{it}}{P_{i,t-1}} \right)
$$

In [920]:
data["Price"] = np.log((data["Price"] + data["Dividends"]) / data["Price"].shift(1))
data = data.drop(columns=["Dividends"])
data.rename(columns={"Price": "Log Return"}, inplace=True)
data

,Ticker,Name,Sector,Date,Log Return,Benchmark,Bond Yield,Log Inflation,Log IPI,Log USD/EUR
0,AC.PA,Accor,Consumer Discretionary,2008-11,NaN,3217.969971,2.70,-0.004796,-0.185435,-0.045293
1,AC.PA,Accor,Consumer Discretionary,2008-12,-0.124660,2973.919922,2.06,-0.002460,0.163142,0.054806
2,AC.PA,Accor,Consumer Discretionary,2009-01,-0.086381,2702.479980,1.32,-0.004186,-0.207272,-0.015795
3,AC.PA,Accor,Consumer Discretionary,2009-02,-0.081113,2807.340088,0.99,0.003864,0.059060,-0.034893
4,AC.PA,Accor,Consumer Discretionary,2009-03,0.206256,3159.850098,0.74,0.001927,0.044075,0.020525
...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,-0.013648,7135.060059,3.68,0.010136,-0.390923,-0.013591
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,0.019103,6885.649902,3.85,-0.005353,0.488359,-0.020847
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,0.026836,7310.770020,3.89,0.001447,0.104945,-0.011415
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,0.107936,7543.180176,3.89,-0.001788,-0.108930,0.022993


Transforming benchmark price to log market return.

$$
r_m,t = \ln \left( \frac{P_{m,t}}{P_{m,t-1}} \right)
$$

In [923]:
data["Benchmark"] = np.log(data["Benchmark"] / data["Benchmark"].shift(1))
data.rename(columns={"Benchmark": "Log Market Return"}, inplace=True)
data

,Ticker,Name,Sector,Date,Log Return,Log Market Return,Bond Yield,Log Inflation,Log IPI,Log USD/EUR
0,AC.PA,Accor,Consumer Discretionary,2008-11,NaN,NaN,2.70,-0.004796,-0.185435,-0.045293
1,AC.PA,Accor,Consumer Discretionary,2008-12,-0.124660,-0.078870,2.06,-0.002460,0.163142,0.054806
2,AC.PA,Accor,Consumer Discretionary,2009-01,-0.086381,-0.095711,1.32,-0.004186,-0.207272,-0.015795
3,AC.PA,Accor,Consumer Discretionary,2009-02,-0.081113,0.038068,0.99,0.003864,0.059060,-0.034893
4,AC.PA,Accor,Consumer Discretionary,2009-03,0.206256,0.118287,0.74,0.001927,0.044075,0.020525
...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,-0.013648,-0.025139,3.68,0.010136,-0.390923,-0.013591
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,0.019103,-0.035581,3.85,-0.005353,0.488359,-0.020847
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,0.026836,0.059909,3.89,0.001447,0.104945,-0.011415
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,0.107936,0.031295,3.89,-0.001788,-0.108930,0.022993


Transform bond yield to log risk-free return.

$$
r_{f,t} = \ln \left( 1 + R_{f,t} \right)
$$

In [926]:
data["Bond Yield"] = np.log(1 + data["Bond Yield"] / 100)
data.rename(columns={"Bond Yield": "Log Risk-Free Return"}, inplace=True)
data

,Ticker,Name,Sector,Date,Log Return,Log Market Return,Log Risk-Free Return,Log Inflation,Log IPI,Log USD/EUR
0,AC.PA,Accor,Consumer Discretionary,2008-11,NaN,NaN,0.026642,-0.004796,-0.185435,-0.045293
1,AC.PA,Accor,Consumer Discretionary,2008-12,-0.124660,-0.078870,0.020391,-0.002460,0.163142,0.054806
2,AC.PA,Accor,Consumer Discretionary,2009-01,-0.086381,-0.095711,0.013114,-0.004186,-0.207272,-0.015795
3,AC.PA,Accor,Consumer Discretionary,2009-02,-0.081113,0.038068,0.009851,0.003864,0.059060,-0.034893
4,AC.PA,Accor,Consumer Discretionary,2009-03,0.206256,0.118287,0.007373,0.001927,0.044075,0.020525
...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08,-0.013648,-0.025139,0.036139,0.010136,-0.390923,-0.013591
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09,0.019103,-0.035581,0.037777,-0.005353,0.488359,-0.020847
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10,0.026836,0.059909,0.038162,0.001447,0.104945,-0.011415
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11,0.107936,0.031295,0.038162,-0.001788,-0.108930,0.022993


Remove dates that are outside the scope.

In [929]:
low = pd.Timestamp("2009-01")
high = pd.Timestamp("2024-02")

data["Date"] = pd.to_datetime(data["Date"])

data = data[(data["Date"] >= low) & (data["Date"] < high)]
data

,Ticker,Name,Sector,Date,Log Return,Log Market Return,Log Risk-Free Return,Log Inflation,Log IPI,Log USD/EUR
2,AC.PA,Accor,Consumer Discretionary,2009-01-01,-0.086381,-0.095711,0.013114,-0.004186,-0.207272,-0.015795
3,AC.PA,Accor,Consumer Discretionary,2009-02-01,-0.081113,0.038068,0.009851,0.003864,0.059060,-0.034893
4,AC.PA,Accor,Consumer Discretionary,2009-03-01,0.206256,0.118287,0.007373,0.001927,0.044075,0.020525
5,AC.PA,Accor,Consumer Discretionary,2009-04-01,0.072753,0.036602,0.007869,0.001496,-0.096637,0.010708
6,AC.PA,Accor,Consumer Discretionary,2009-05-01,-0.034363,-0.042764,0.006876,0.001600,-0.095463,0.034291
...,...,...,...,...,...,...,...,...,...,...
7083,VIV.PA,Vivendi,Consumer Discretionary,2023-08-01,-0.013648,-0.025139,0.036139,0.010136,-0.390923,-0.013591
7084,VIV.PA,Vivendi,Consumer Discretionary,2023-09-01,0.019103,-0.035581,0.037777,-0.005353,0.488359,-0.020847
7085,VIV.PA,Vivendi,Consumer Discretionary,2023-10-01,0.026836,0.059909,0.038162,0.001447,0.104945,-0.011415
7086,VIV.PA,Vivendi,Consumer Discretionary,2023-11-01,0.107936,0.031295,0.038162,-0.001788,-0.108930,0.022993


Convert ticker and date as indexes.

In [932]:
data["Date"] = pd.to_datetime(data["Date"]).dt.strftime('%Y-%m')
data = data.set_index(["Ticker", "Date"])
data

C:\Users\Admin\AppData\Local\Temp\ipykernel_16912\3630494879.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Name                  Sector  Log Return  \
Ticker Date                                                   
AC.PA  2009-01    Accor  Consumer Discretionary   -0.086381   
       2009-02    Accor  Consumer Discretionary   -0.081113   
       2009-03    Accor  Consumer Discretionary    0.206256   
       2009-04    Accor  Consumer Discretionary    0.072753   
       2009-05    Accor  Consumer Discretionary   -0.034363   
...                 ...                     ...         ...   
VIV.PA 2023-08  Vivendi  Consumer Discretionary   -0.013648   
       2023-09  Vivendi  Consumer Discretionary    0.019103   
       2023-10  Vivendi  Consumer Discretionary    0.026836   
       2023-11  Vivendi  Consumer Discretionary    0.107936   
       2023-12  Vivendi  Consumer Discretionary    0.077432   

                Log Market Return  Log Risk-Free Return  Log Inflation  \
Ticker Date                                                              
AC.PA  2009-01          -0.095711              0.013114      -0.004186   
       2009-02           0.038068              0.009851       0.003864   
       2009-03           0.118287              0.007373       0.001927   
       2009-04           0.036602              0.007869       0.001496   
       2009-05          -0.042764              0.006876       0.001600   
...                           ...                   ...            ...   
VIV.PA 2023-08          -0.025139              0.036139       0.010136   
       2023-09          -0.035581              0.037777      -0.005353   
       2023-10           0.059909              0.038162       0.001447   
       2023-11           0.031295              0.038162      -0.001788   
       2023-12           0.014944              0.038162       0.001448   

                 Log IPI  Log USD/EUR  
Ticker Date                            
AC.PA  2009-01 -0.207272    -0.015795  
       2009-02  0.059060    -0.034893  
       2009-03  0.044075     0.020525  
       2009-04 -0.096637     0.010708  
       2009-05 -0.095463     0.034291  
...                  ...          ...  
VIV.PA 2023-08 -0.390923    -0.013591  
       2023-09  0.488359    -0.020847  
       2023-10  0.104945    -0.011415  
       2023-11 -0.108930     0.022993  
       2023-12 -0.070303     0.008735  

[7012 rows x 8 columns]

Export the data for further analysis.

In [935]:
data.to_csv("data/data.csv", index=True, sep=";")